In [ ]:
import matplotlib.pyplot as plt
import random

In [1]:
import gillespie

ModuleNotFoundError: No module named 'gillespie'

In [ ]:
alpha = 1  # reproduction rate of prey
beta = 0.01  # prey consumption rate (and reproduction of predator)
gamma = 2  # death rate of predators
t = 15  # duration

reag_quant = [50, 50]  # prey, predators

propensities = [lambda prey, pred: alpha * prey,  # prey birth
                lambda prey, pred: beta * prey * pred,  # prey death / predator birth
                lambda prey, pred: gamma * pred]  # predator death

state_change_vect = [[1, 0],  # prey birth
                 [-1, 1],  # prey death / predator birth
                 [0, -1]]  # predator death

In [ ]:
# data for schogle |  WORKING!!!!!!!!!!!!!!!!!!!!!!

reag_quant = [100000, 200000, 250]
state_change_vect = [[-1, 0, +1],
                     [+1, 0, -1],
                     [0, -1, +1],
                     [0, +1, -1]]
combinatorics = [lambda a,b,x : (3*10e-8*a*x*(x-1)/2),
                lambda a,b,x : (0.0001*x*(x-1)*(x-2)/6),
                lambda a,b,x : b*0.001,
                lambda a,b,x : x*3.5]

In [ ]:
result_dict ={}
for epoch in range(100):
    
    a = gillespie_ssa(reagent_quantity=reag_quant, state_change_vectors=state_change_vect, combinatorics=combinatorics, iteration=100000, set_fixed_reagents=[0,1])
    result_dict[epoch] = [a.timestep_list, a.molecular_species_history]
    plt.plot([x for x in range(a.actual_iteration+1)],[y[2] for y in a.molecular_species_history])
    #plt.plot([x for x in a.timestep_list],[y[1] for y in a.molecular_species_history])